## Generazione nuovi esempi di training

In [9]:
import os
import pandas as pd
dir_path='../../Structured/itunes-amazon/exp1'
tableA_df = pd.read_csv(os.path.join(dir_path,'tableA.csv'))
tableB_df = pd.read_csv(os.path.join(dir_path,'tableB.csv'))

In [10]:
tableA_df['Artist_Name']= tableA_df['Artist_Name'].apply(lambda s: s.lower())
tableA_df['Song_Name']= tableA_df['Song_Name'].apply(lambda s: s.lower())

In [3]:
tableB_df['Artist_Name']= tableB_df['Artist_Name'].apply(lambda s: s.lower())
tableB_df['Song_Name']= tableB_df['Song_Name'].apply(lambda s: s.lower())

In [4]:
def minimumEditDistance(s1,s2):
    if len(s1) > len(s2):
        s1,s2 = s2,s1
    distances = range(len(s1) + 1)
    for index2,char2 in enumerate(s2):
        newDistances = [index2+1]
        for index1,char1 in enumerate(s1):
            if char1 == char2:
                newDistances.append(distances[index1])
            else:
                newDistances.append(1 + min((distances[index1],
                                             distances[index1+1],
                                             newDistances[-1])))
        distances = newDistances
    return distances[-1]

In [5]:
def jaccard_distance(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    #print(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float((union-intersection) / union)

In [6]:
import nltk
stopwords = nltk.download("stopwords")

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

[nltk_data] Downloading package stopwords to /home/nvidia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def get_clean_tokens(s):
    tokens =tokenizer.tokenize(s)
    return [word for word in tokens if word not in stop_words]

## Ricerca di canzoni non matching tramite jaccard similarity

In [20]:
import random as rd

In [49]:
def generateNewNegatives(source1,source2,nsamples):
    newsamples = []
    while(len(newsamples)<nsamples):
        k = rd.randint(0,source1.shape[0]-1)
        l_row = source1.iloc[k]
        ltime = l_row['Time']
        lartist_cleaned = get_clean_tokens(l_row['Artist_Name'])
        lsong_cleanead = get_clean_tokens(l_row['Song_Name'])
        source2_sametime = source2[source2.Time==ltime]
        ##per avere più variabilità
        for idx in source2_sametime.index[0:5]:
            current_rrow = source2.iloc[idx]
            rsong_cleaned = get_clean_tokens(current_rrow['Song_Name'])
            rartist_clean = get_clean_tokens(current_rrow['Artist_Name'])
            if (jaccard_distance(lartist_cleaned,rsong_cleaned)>=0.7 or 
                jaccard_distance(lsong_cleanead,rsong_cleaned) >=0.7):
                newsamples.append((l_row['id'],current_rrow['id'],0))
    return newsamples

In [50]:
newSamples = generateNewNegatives(tableA_df,tableB_df,30)
newSamples_df = pd.DataFrame(data = newSamples,columns=['ltable_id','rtable_id','label'])

In [57]:
newSamples_df.head(10)

,ltable_id,rtable_id,label
0,2152,206,0
1,2152,220,0
2,2152,307,0
3,2152,311,0
4,2152,836,0
5,3936,501,0
6,3936,2160,0
7,3936,2577,0
8,3936,3129,0
9,3936,4297,0


In [58]:
tableA_df.iloc[2152]

id                                      2152
Song_Name      goodnight , hollywood blvd. .
Artist_Name                       ryan adams
Album_Name                              Gold
Genre                           Rock , Music
Price                                 $ 1.29
CopyRight         2014 UMG Recordings , Inc.
Time                                    3:24
Released                           30-Sep-14
Name: 2152, dtype: object

In [59]:
tableB_df.iloc[220]

id                                                      220
Song_Name              Watch The Sun Come Up ( Radio Edit )
Artist_Name                                         Example
Album_Name                               #Hits [ Explicit ]
Genre                                    Dance & Electronic
Price                                                $ 0.99
CopyRight      ( c ) 2013 Ministry of Sound Recordings Ltd.
Time                                                   3:24
Released                                    August 5 , 2013
Name: 220, dtype: object

In [60]:
tableA_df.iloc[3936]['Time']

'5:13'

In [61]:
tableB_df.iloc[4297]['Time']

'5:13'

In [52]:
newSamples_df.to_csv('exp26/extended_train10perc.csv',index=False)

In [51]:
!mkdir exp25
train_df = pd.read_csv('../../Structured/itunes-amazon/exp1/train.csv')
train_df.shape

(321, 3)

In [53]:
new_train_df = pd.concat([train_df,newSamples_df])

In [55]:
new_train_df.to_csv('exp26/extended_train10perc.csv',index=False)